In [1]:
from opik import configure
from opik.integrations.langchain import OpikTracer

configure(api_key="dHhF5jDOpifAMNJH9UalSjR0o", workspace="yu-li")
opik_tracer = OpikTracer(project_name="sku_match")


OPIK: Opik is already configured. You can check the settings by viewing the config file at /Users/liyu/.opik.config
OPIK: Configuration completed successfully. Traces will be logged to 'Default Project' project. To change the destination project, see: https://www.comet.com/docs/opik/tracing/log_traces#configuring-the-project-name


In [2]:
# 使用langgraph接入大模型，使用大模型判断是否相似
# 输入商品名称和top3商品名称，输入相似的结果
from langchain_openai import ChatOpenAI
import os

# os.environ["LANGSMITH_API_KEY"] = "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"
# os.environ["LANGSMITH_PROJECT"] = "rag"
# os.environ["LANGSMITH_TRACING"] = "true"


qwen3_8B = ChatOpenAI(
    model="Qwen/Qwen3-8B",
    base_url="https://api.siliconflow.cn/v1",
    api_key="sk-idaudysppselrwglygkbtatkregsbxhaxypaeulbfpavrals",
)


In [3]:
from pydantic import BaseModel, Field
from typing_extensions import TypedDict


class RankSelect(BaseModel):
    """Model for match top rank. Provide json constraints."""

    match_index: int = Field(
        description="返回匹配 origin_product 商品的候选商品索引. 如果是top1_candidate返回1; 如果是top2_candidate返回2; 如果是top3_candidate返回3；不匹配或其他情况返回0。",
    )
    match_reason: str = Field(
        description="返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。",
    )

In [4]:
rank_prompt = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品描述中选择描述一致的索引值。

<Task>
你需要按照下面步骤进行思考处理和返回：
1. 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为origin_product的候选匹配商品信息。
2. 每个商品信息均包含品牌、描述、商品名、规格数量4部分信息，例如输入“百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋”，可提取品牌为百事、描述为意大利香浓红烩味原切马铃薯片、商品名为薯片、规格数量为40g_袋。
3. 首先提取 origin_product 输入商品的品牌、描述、商品名、规格数量信息。
4. 依次对输入的候选商品信息 top1_candidate,top2_candidate,top3_candidate 依次提取商品的品牌、描述、商品名、规格数量信息并和origin_product提取信息按照4部分进行匹配比对。
5. 全面分析思考，选出与origin_product描述一致的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。
</Task>

以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

<Guidelines>
以下是origin_product和候选商品不匹配的举例情况，任意一种情况不匹配，当前判断的候选商品则不匹配：
1. 商品品牌不一致，例如origin_product识别品牌为可口，candidate识别为品牌为百事；评判时要求商品品牌不必完全相同，部分品牌会带有商品信息，如乌苏啤酒;语义相同也视为一致，例如乌苏与乌苏啤酒视为品牌一致。
2. 商品数量规格不一致，origin_product识别规格为250ml,candidate识别规格为300ml，商品数量规则匹配需要满足数量匹配且规格大小匹配；规格可包括L、kg、千克、盒、件、片、条等市面计量单位，如没有说明默认数量为1；评判时注意抑语义相同及规格一致，如1L和1000ml为一致；个、袋、盒、瓶等单个包装规格在整体语义下也可一致。
3. 商品名语义不一致，origin_product识别为识字卡，candidate识别为挪车卡；评断时名称不同但语义相同时视为一致，对比双方完全不是同一类型商品才会判断为不一致。
4. 商品描述完全不一致，origin_product识别描述冬季保暖羽绒服，candidate识别描述为夏季防晒衣；评判时要求语义偏移或不同不视为不一致，只有完全冲突时才视为不匹配，如冬天与夏天，辣味与甜味。
</Guidelines>

<Output Format>
请使用以下键值以有效的 JSON 格式进行响应：
"match_index": int. 返回匹配 origin_product 商品的候选商品索引. 如果是top1_candidate返回1; 如果是top2_candidate返回2; 如果是top3_candidate返回3；不匹配或其他情况返回0。,
"match_reason":str. 返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。
</Output Format>
"""

In [5]:
from LLM import longcat_flash_chat, GLM4_9B, qwen3_14b, glm4_5_air, qwen3_next_80b,qwen3_30b_instruct, glm_z1_9b, \
    qwen3_235b_thinking, qwen3_30b_thinking

model_qwen_8b = qwen3_8B.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_glm_9b = GLM4_9B.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_glm_z1_9b = glm_z1_9b.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_qwen3_14b = qwen3_14b.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_qwen3_30b_thinking = qwen3_30b_thinking.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_qwen3_30b = qwen3_30b_instruct.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_qwen3_next_80b = qwen3_next_80b.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_glm4_5_air = glm4_5_air.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_qwen3_235b_thinking = qwen3_235b_thinking.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)
model_longcat_flash = longcat_flash_chat.with_structured_output(RankSelect).with_retry(stop_after_attempt=3)


In [6]:
# origin_product = "达利园 香奶味软面包  200g／袋 面包就要达利园"
# top1_candidate="达利园 香奶味法式软面包 360g/袋"
# top2_candidate="达利园 桂圆莲子八宝粥 360g/罐"
# top3_candidate="洽洽 香瓜子 160g/袋"
# top1_candidate = ""
# top2_candidate = "达利园 香奶味法式软面包 360g/袋"
# top3_candidate = ""

In [22]:
origin_product="东鹏特饮 东鹏特饮维生素功能饮料 500ml_瓶"
top1_candidate="东鹏  维生素能量饮料 500ml"
top2_candidate=""
top3_candidate=""

In [9]:
origin_product = "迪味臻 五香卤蛋 30g_袋"
top1_candidate = "迪味臻 五香卤蛋 30g_个"
top2_candidate = ""
top3_candidate = ""

In [37]:
origin_product = "乌苏啤酒 11°P红乌苏啤酒 620ml_瓶"
top1_candidate = "乌苏啤酒 11°P夺命大乌苏啤酒 620ml_瓶"
top2_candidate = ""
top3_candidate = ""

In [ ]:
origin_product = "禛香 禛香肥牛味大豆素肉大豆粕膨化制品 18g_袋"
top1_candidate = "禛香 牛肉味素肉辣条 儿时怀旧解馋小零食小吃休闲食品辣条肥牛味 18g_袋"
top2_candidate = ""
top3_candidate = ""

In [43]:
origin_product = "苏菲SOFY M码超薄款超熟睡安心裤裤型卫生巾 2条_包"
top1_candidate = "【5包特惠】苏菲SOFY M码超薄款超熟睡安心裤裤型卫生巾 2条_包5"
top2_candidate = "苏菲 XL码超薄款超熟睡安心裤 2条5包"
top3_candidate = "【苏菲】超熟睡安心裤裤型卫生巾M号 2条_袋"

In [7]:
origin_product = "娃哈哈AD钙奶儿童奶含乳饮料饮品童年哇哈哈 220gx4瓶／排"
top1_candidate = "【2排】娃哈哈 AD钙奶饮料 220g4瓶2排"
top2_candidate = "【4瓶装】娃哈哈 AD钙奶饮料 220g4瓶_排"
top3_candidate = "【整箱】娃哈哈 AD钙奶饮料 220g24瓶_箱"

In [8]:
prompt_format = rank_prompt.format(origin_product=origin_product, top1_candidate=top1_candidate,
                                   top2_candidate=top2_candidate, top3_candidate=top3_candidate)
prompt_format

'\n你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品描述中选择描述一致的索引值。\n\n<Task>\n你需要按照下面步骤进行思考处理和返回：\n1. 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为origin_product的候选匹配商品信息。\n2. 每个商品信息均包含品牌、描述、商品名、规格数量4部分信息，例如输入“百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋”，可提取品牌为百事、描述为意大利香浓红烩味原切马铃薯片、商品名为薯片、规格数量为40g_袋。\n3. 首先提取 origin_product 输入商品的品牌、描述、商品名、规格数量信息。\n4. 依次对输入的候选商品信息 top1_candidate,top2_candidate,top3_candidate 依次提取商品的品牌、描述、商品名、规格数量信息并和origin_product提取信息按照4部分进行匹配比对。\n5. 全面分析思考，选出与origin_product描述一致的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。\n</Task>\n\n以下是输入信息：\n<products_info>\norigin_product：娃哈哈AD钙奶儿童奶含乳饮料饮品童年哇哈哈 220gx4瓶／排\ntop1_candidate：【2排】娃哈哈 AD钙奶饮料 220g4瓶2排\ntop2_candidate：【4瓶装】娃哈哈 AD钙奶饮料 220g4瓶_排\ntop3_candidate：【整箱】娃哈哈 AD钙奶饮料 220g24瓶_箱\n</products_info>\n\n<Guidelines>\n以下是origin_product和候选商品不匹配的举例情况，任意一种情况不匹配，当前判断的候选商品则不匹配：\n1. 商品品牌不一致，例如origin_product识别品牌为可口，candidate识别为品牌为百事；评判时要求商品品牌不必完全相同，部分品牌会带有商品信息，如乌苏啤酒;语义相同也视为一致，例如乌苏与乌苏啤酒视为品牌一致。\

In [11]:
from langchain_core.messages import (
    AIMessage,
    HumanMessage,
)

# model_qwen_8b 规格错误
# model_glm_9b 正确但不稳定
# model_glm_z1_9b 规格错误
# model_qwen3_14b 正确但不稳定
# model_qwen3_30b_thinking 指定遵循失败
# model_qwen3_30b 正确
# model_qwen3_next_80b 正确
# model_glm4_5_air 正确
# model_qwen3_235b_thinking json模式指定遵循失败
# model_longcat_flash 正确，分析准确
rankSelect = model_glm_9b.invoke([HumanMessage(content=prompt_format)],
                               config={"callbacks": [opik_tracer]})


In [10]:
rankSelect

RankSelect(match_index=2, match_reason='首先对 origin_product 进行信息提取：品牌为‘娃哈哈’，描述为‘儿童奶含乳饮料饮品童年哇哈哈’，商品名为‘AD钙奶’，规格数量为‘220gx4瓶／排’。接着依次分析候选商品：\n\n- top1_candidate：品牌为‘娃哈哈’（一致）；描述为‘AD钙奶饮料’（与原描述中‘儿童奶含乳饮料饮品童年哇哈哈’语义相近，均指向娃哈哈AD钙奶饮品，核心功能和品类一致）；商品名可视为‘AD钙奶’（一致）；规格数量为‘220g4瓶2排’，其中‘220g’与原规格‘220g’一致，‘4瓶’也一致，但包装单位为‘2排’而非‘1排’，存在数量倍数差异（原为1排4瓶，候选为2排），因此整体规格数量不匹配（2排 ≠ 1排），故不匹配。\n\n- top2_candidate：品牌为‘娃哈哈’（一致）；描述为‘AD钙奶饮料’（与原描述语义一致，均为娃哈哈的儿童含乳饮料）；商品名‘AD钙奶’（一致）；规格数量为‘220g4瓶_排’，即每排4瓶，220g/瓶，与origin_product的‘220gx4瓶／排’完全一致，仅表达方式略有不同（‘_排’与‘／排’为同义符号），属于同一规格，因此规格数量匹配。综合四部分信息完全一致，判定匹配。\n\n- top3_candidate：品牌为‘娃哈哈’（一致）；描述为‘AD钙奶饮料’（语义一致）；商品名‘AD钙奶’（一致）；但规格数量为‘220g24瓶_箱’，即每箱24瓶，与原规格‘4瓶／排’在数量和包装单位上均不一致（4瓶 vs 24瓶，排 vs 箱），且未体现‘4瓶/排’的组合形式，因此不匹配。\n\n综上，只有 top2_candidate 在品牌、描述、商品名、规格数量四方面均一致，故返回索引2。')

In [ ]:
rank_prompt_v1 = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品描述中选择描述一致的编号值。
你需要按照下面步骤进行思考处理和返回：
- 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为origin_product的候选匹配商品信息。
- 每个商品信息均包含品牌、描述、商品名、规格数量4部分信息，例如输入“百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋”，可提取品牌为百事、描述为意大利香浓红烩味原切马铃薯片、商品名为薯片、规格数量为40g_袋。
- 首先提取 origin_product 输入商品的品牌、描述、商品名、规格数量信息。
- 依次对输入的候选商品信息 top1_candidate,top2_candidate,top3_candidate 依次提取商品的品牌、描述、商品名、规格数量信息并和origin_product提取信息按照4部分进行匹配比对。
- 全面分析思考，选出与origin_product描述一致的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。

以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

以下是origin_product和候选商品不匹配的举例情况，任意一种情况不匹配，当前判断的候选商品则不匹配：
1. candidate信息为空，当前判断的候选商品不匹配；评判时当前候选商品信息为空可跳过，全部候选商品为空时直接返回0。
2. 商品品牌不一致，例如origin_product识别品牌为可口，candidate识别为品牌为百事；评判时要求商品品牌不必完全相同，部分品牌会带有商品信息，如乌苏啤酒，语义相同也视为一致，例如乌苏与乌苏啤酒视为品牌一致。
3. 商品数量规格不一致，origin_product识别规格为250ml,candidate识别规格为300ml，商品数量规则匹配需要满足数量匹配且规格大小匹配；规格可包括L、kg、千克、盒、件、片、条等市面计量单位，如没有说明默认数量为1；评判时注意抑语义相同及规格一致，如1L和1000ml为一致；个、袋、盒等单个规格在整体语义下也可一致。
4. 商品名语义不符合，origin_product识别为识字卡，candidate识别为挪车卡。
5. 商品描述有冲突，origin_product识别描述冬季保暖，candidate识别描述为夏季防晒；评判时要求语义偏移或不同不视为不一致，只有完全冲突时才视为不匹配，如冬天与夏天，辣味与甜味。
6. 商品信息整体语义冲突，origin_product商品信息为小型便携式冲牙器，candidate商品信息为家用取暖器暖风机；只有比较双方商品语义完全不符合同一类商品才判断为冲突。

请使用以下键值以有效的 JSON 格式进行响应：
"match_index": int. 返回匹配 origin_product 商品的候选商品编号. 如果是 top1_candidate 返回 1; 如果是 top2_candidate 返回 2; 如果是 top3_candidate 返回 3；不匹配或其他情况返回0。,
"match_reason":str. 返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。
"""

In [ ]:
rank_prompt_v2 = """
你是一个电商运营专家，请你根据输入的商品描述信息在多个候选商品描述中选择描述一致的编号值。
你需要按照下面步骤进行思考处理和返回：
- 接收 origin_product、top1_candidate、top2_candidate、top3_candidate 四个商品信息描述，origin_product 为原始商品信息、top*_candidate 为origin_product的候选匹配商品信息。
- 每个商品信息均包含品牌、描述、商品名、规格数量4部分信息，例如输入“百事食品乐事 意大利香浓红烩味原切马铃薯片 40g_袋”，可提取品牌为百事、描述为意大利香浓红烩味原切马铃薯片、商品名为薯片、规格数量为40g_袋。
- 首先提取 origin_product 输入商品的品牌、描述、商品名、规格数量信息。
- 依次对输入的候选商品信息 top1_candidate,top2_candidate,top3_candidate 依次提取商品的品牌、描述、商品名、规格数量信息并和origin_product提取信息按照4部分进行匹配比对。
- 全面分析思考，选出与origin_product描述一致的候选商品编号（如，1，2，3），需要特别注意，如果候选商品与origin_product都不匹配，请返回0。

以下是输入信息：
<products_info>
origin_product：{origin_product}
top1_candidate：{top1_candidate}
top2_candidate：{top2_candidate}
top3_candidate：{top3_candidate}
</products_info>

以下是origin_product和候选商品不匹配的举例情况，任意一种情况不匹配，当前判断的候选商品则不匹配：
1. 商品品牌不一致，例如origin_product识别品牌为可口，candidate识别为品牌为百事；评判时要求商品品牌不必完全相同，部分品牌会带有商品信息，如乌苏啤酒;语义相同也视为一致，例如乌苏与乌苏啤酒视为品牌一致。
2. 商品数量规格不一致，origin_product识别规格为250ml,candidate识别规格为300ml，商品数量规则匹配需要满足数量匹配且规格大小匹配；规格可包括L、kg、千克、盒、件、片、条等市面计量单位，如没有说明默认数量为1；评判时注意抑语义相同及规格一致，如1L和1000ml为一致；个、袋、盒、瓶等单个包装规格在整体语义下也可一致。
3. 商品名语义不一致，origin_product识别为识字卡，candidate识别为挪车卡；评断时名称不同但语义相同时视为一致，对比双方完全不是同一类型商品才会判断为不一致。
4. 商品描述完全不一致，origin_product识别描述冬季保暖羽绒服，candidate识别描述为夏季防晒衣；评判时要求语义偏移或不同不视为不一致，只有完全冲突时才视为不匹配，如冬天与夏天，辣味与甜味。

请使用以下键值以有效的 JSON 格式进行响应：
"match_index": int. 返回匹配 origin_product 商品的候选商品编号. 如果是 top1_candidate 返回 1; 如果是 top2_candidate 返回 2; 如果是 top3_candidate 返回 3；不匹配或其他情况返回0。,
"match_reason":str. 返回匹配结果过程的思考原因，需要体现对输入信息的处理、比对、分析等过程。
"""